# 1. Entorno

In [1]:
#Creamos un Notebook en Python sobre JupyterLab

# 2. Librerías

In [1]:
#Utilitario para construir modelos de Machine Learning y Deep Learning
import sagemaker

#Librería para manipular los servicios de AWS
import boto3

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


# 3. Inicio de Sesión sobre SageMaker

In [2]:
#Iniciamos sesión en el servicio de SageMaker
sesion = sagemaker.Session()

#Obtenemos la ejecución en donde estamos trabajando
region = sesion.boto_region_name

#Obtenemos el rol de ejecución de SageMaker
#El rol de ejecución permite a SageMaker usar otros servicios de AWS
rol = sagemaker.get_execution_role()

# 4. Lectura del Modelo

In [3]:
#Definimos el nombre del entrenamiento al que nos conectamos
nombreDeEntrenamiento = "linear-learner-240927-1556-012-966d28e8"

#Definimos el algoritmo que usamos para entrenar el modelo
algoritmo = "linear-learner"

#Nos conectamos al servicio de SageMaker
sagemakerCliente = boto3.client("sagemaker")

#Obtenemos la descripción del entrenamiento
descripcionDeEntrenamiento = sagemakerCliente.describe_training_job(TrainingJobName = nombreDeEntrenamiento)

#Obtenemos la ruta en donde el modelo se encuentra almacenado
rutaDelModelo = descripcionDeEntrenamiento["ModelArtifacts"]["S3ModelArtifacts"]

#Verificamos
print(rutaDelModelo)

#Utilitario para leer modelos
from sagemaker.model import Model
#Leemos el modelo
modelo = Model(
    model_data = rutaDelModelo, #Ruta del modelo
    role = rol, #Rol de ejecución
    image_uri = sagemaker.image_uris.retrieve(algoritmo, region), #Descargamos la implementación del algoritmo desde la región donde entrenamos
    sagemaker_session = sesion #Sesión de SageMaker
)

s3://sagemaker-us-east-1-767828747956/linear-learner-240927-1556-012-966d28e8/output/model.tar.gz


# 5. Creación de Endpoint del modelo

In [4]:
#Endpoint de acceso al modelo
#IMPORTANTE: En "XXX" colocar la fecha de hoy, hay un bug que hace que no puedas ver tu modelo si previamente ya lo creaste y borraste
nombreDelEndpoint = "endpoint-numerico-27-09-2024"

In [5]:
#Tipo de instancias en donde se publica el modelo
tipoDeInstanciaDeEndpoint = "ml.m5.large"

In [6]:
#Cantidad de servidores iniciales en donde se publica el modelo para ser usado
cantidadInicialDeInstancias = 1

In [7]:
#Desplegamos el modelo
#TIEMPO: 5 MINUTOS
modelo.deploy(
    initial_instance_count = cantidadInicialDeInstancias, #Cantidad de servidores
    instance_type = tipoDeInstanciaDeEndpoint, #Tipo de servidor
    endpoint_name = nombreDelEndpoint #Nombre del punto de acceso al modelo
)

--------!